In [1]:
import sys
sys.path.insert(0, "/Users/aimachine/PyImage/utils")
import numpy as np
import tensorflow as tf
import importlib
import matplotlib.pyplot as plt
import os
import glob
import skimage
import cv2
import time
from skimage import measure
from skimage import filters
from sklearn.cluster import DBSCAN
from skimage.filters import threshold_otsu
from tifffile import imread
import scipy
from PIL import Image
from skimage.filters.rank import median
from scipy import ndimage
from scipy.ndimage.filters import median_filter
from DBScan import segment_dbscan,show_clusters_DBSCAN
from DiscreteClustering import Discretize_Clustering
from Normalize import normalizeFloat, normalizeMinMax, Path, save_tiff_imagej_compatible

/Users/aimachine/anaconda3/envs/tensorflowpy3pt5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/aimachine/anaconda3/envs/tensorflowpy3pt5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/aimachine/anaconda3/envs/tensorflowpy3pt5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/aimachine/anaconda3/envs/tensorflowpy3pt5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/aimachine/anaconda3/envs/tensorflowpy3pt5/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed

In [2]:
basedir = '/Users/aimachine/Documents/JuliaData/Julia/Subset/OzRawMedian/Boundary/'
targetdir = '/Users/aimachine/Documents/JuliaData/Julia/Subset/OzRawMedian/Result/'

In [11]:
Path = os.path.join(basedir, '*.tif')
X = []
Names = []
filesRaw = glob.glob(Path)
maxtime = 0
axes = 'TYX'
for fname in filesRaw:
      x = imread(fname)
      min = 0
      max = 255
      x = normalizeMinMax(x, min, max)  
      
      X.append(x)
    
     
      Names.append(fname)
      if X[0].shape[0] > maxtime:
        maxtime = X[0].shape[0]
X.sort
Names.sort


ReshapeX = []
for i in range(len(X)):
    y = np.zeros((maxtime, X[0].shape[1], X[0].shape[2]))
    image = X[i]
    
    y[:X[i].shape[0],:,:] = image[:,:,:]
    ReshapeX.append(y)
    
ReshapeX.sort

print('Total number of input images = ', len(X))
print('Image size = ', X[0].shape)
print('Time dim = ', maxtime)

timerange = maxtime

doDBscan = True
doClustering = False

Total number of input images =  1
Image size =  (241, 321, 314)
Time dim =  241


In [12]:
def Clustering(img):
    Labels = img
    T_REGIONS = []
    for j in range(timerange):
      smallimg = img[j,:,:]
      
      """Put clustering code"""
      
      labels, N_REGIONS =Discretize_Clustering(smallimg, N_REGIONS = 150)
      Labels[j,:,:] = labels
      T_REGIONS.append( N_REGIONS)  
    return Labels, T_REGIONS       

    
       
    
    

In [13]:
def DBScan(img):
    
    
    
    segmentation_params = {
        "IntensityThreshold": 45,
        "BlurKernelSize": 3,
        "BlurSigma": 1,

        "Clustering": {
            "Eps": 2,
            "MinDensity": 70,
            "BorderPoints": 1,

            "KdTree": {
                "BucketSize": 20,
                "SplitRule": 4,
                "Approx": 0
            }
        },
    }
    Labels = img
    Clusters = []
    for j in range(timerange):
      smallimg = img[j,:,:]
      
      #medianimg = median_filter(smallimg, size = 2)
      cluster_list, cluster_image = segment_dbscan(smallimg, segmentation_params)
      Labels[j,:,:] = cluster_image;  
      Clusters.append(cluster_list)
    return Labels, Clusters     

In [14]:
for i in range(0, len(ReshapeX)):
  newimg = ReshapeX[i]
  Clusterimg = newimg;
  t0 = time.time()
  if doDBscan:
    Labels, Clusters = DBScan(newimg)
    t1 = time.time()
    print('time taken', t1 - t0)
    save_tiff_imagej_compatible((targetdir + 'SuperPixel' + os.path.basename(Names[i])) , Labels, axes)
    
    
  if doClustering:
    Labels, T_REGIONS = Clustering(newimg)
    

ValueError: could not broadcast input array from shape (0) into shape (321,314)

In [ ]:
imagenumber = 0;
if imagenumber > maxtime:
   imagenumber = maxtime - 1
if doDBscan:
    plt.figure(1)
    
    label = Labels[0]
    img = newimg[0,:,:]
    labelimg = label[:,:]    
    clusterimg = show_clusters_DBSCAN(img, Clusters[0])
    
    plt.imshow(labelimg)
    plt.show() 
if doClustering:
    plt.figure(figsize=(5, 5))
    plt.imshow(newimg[0,:,:])
    plt.figure(figsize=(5, 5))
    plt.imshow(newimg[0,:,:])
    N_REGIONS = T_REGIONS[0]
    labels = Labels[0,:,:]
    for l in range(N_REGIONS):
        
        plt.contour(labels == l, contours=1,
                    colors=[plt.cm.nipy_spectral(l / float(N_REGIONS)), ])
        
